Experiment 7

In [24]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
from torchvision import datasets
from torchvision.transforms import v2


def compute_mean_std(dataset):
    """Computing mean and std of the dataset"""
    loader = DataLoader(dataset, batch_size=32)
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0

    for data, _ in loader:
        # Mean over batch, height, and width, for each channel
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = torch.sqrt(channels_squared_sum / num_batches - mean ** 2)

    return mean, std


#load raw training dataset (without normalization) to compute stats
temp_train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=v2.Compose([
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True)
    ])
)



#Transforms with normalization and data augmentation from exp5
train_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=train_mean, std=train_std),
    # Data augmentation
    v2.RandomCrop(size=(28, 28), padding=4),
    v2.RandomRotation(degrees=(-10, 10)),
    v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),
])

test_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=train_mean, std=train_std)
])

#Datasets and loaders
full_train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=train_transform
)

test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=test_transform
)

train_dataset = full_train_dataset

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


#CNN from exp6
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


#Training
learning_rate = 1e-3
num_epochs = 5

model = CNN().to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

train_losses = []
train_accuracies = []
test_accuracies = []



start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm.tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_losses.append(avg_loss)
    train_accuracies.append(train_acc)

    #Evaluating on test set
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)

    test_acc = correct_test / total_test
    test_accuracies.append(test_acc)

    print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {total_loss:.4f}, Train Acc: {train_acc:.2%}, Test Acc: {test_acc:.2%}")

end_time = time.time()
total_time = end_time - start_time
print(f"\nTotal training time for Experiment 7: {total_time:.2f} seconds")

print(f"Final Test Accuracy for Experiment 7: {test_accuracies[-1]:.4f}")


Epoch 1/5: 100%|██████████████████████████████| 938/938 [00:41<00:00, 22.77it/s]


Epoch 1/5 - Train Loss: 692.1147, Train Acc: 72.21%, Test Acc: 81.57%


Epoch 2/5: 100%|██████████████████████████████| 938/938 [00:40<00:00, 23.21it/s]


Epoch 2/5 - Train Loss: 515.5341, Train Acc: 79.53%, Test Acc: 84.66%


Epoch 3/5: 100%|██████████████████████████████| 938/938 [00:40<00:00, 23.08it/s]


Epoch 3/5 - Train Loss: 449.9911, Train Acc: 82.34%, Test Acc: 86.35%


Epoch 4/5: 100%|██████████████████████████████| 938/938 [00:40<00:00, 23.11it/s]


Epoch 4/5 - Train Loss: 415.8026, Train Acc: 83.61%, Test Acc: 85.40%


Epoch 5/5: 100%|██████████████████████████████| 938/938 [00:41<00:00, 22.84it/s]


Epoch 5/5 - Train Loss: 391.9721, Train Acc: 84.40%, Test Acc: 88.01%

Total training time for Experiment 7: 212.77 seconds
Final Test Accuracy for Experiment 7: 0.8801


Experiment 8

In [30]:
import time
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import v2
from torchvision.models import resnet18, ResNet18_Weights
import tqdm



def compute_mean_std(dataset):
    """Computing mean and std of the dataset."""
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0

    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = torch.sqrt(channels_squared_sum / num_batches - mean ** 2)
    return mean, std


# raw training set to get stats 
temp_train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=v2.Compose([
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
    ])
)


# 3-channel stats for ResNet
mean_val = float(train_mean.item())
std_val = float(train_std.item())
mean_3c = [mean_val, mean_val, mean_val]
std_3c = [std_val, std_val, std_val]


# data augmentation from Q5

train_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Lambda(lambda x: x.repeat(3, 1, 1)),               
    v2.Normalize(mean=mean_3c, std=std_3c),
    v2.RandomCrop(size=(28, 28), padding=4),
    v2.RandomRotation(degrees=(-10, 10)),
    v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),
])

test_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Lambda(lambda x: x.repeat(3, 1, 1)),
    v2.Normalize(mean=mean_3c, std=std_3c),
])

#datasets and loaders

train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=train_transform,
)

test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=test_transform,
)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



#Pre-trained ResNet and configurable fully connected layers


base_model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)


backbone_in_features = base_model.fc.in_features 


for param in base_model.parameters():
    param.requires_grad = False


base_model.fc = nn.Identity()


class PretrainedFashionNet(nn.Module):
    """
    Pre-trained ResNet + configurable fully connected classifier.
    only the newly added fully connected layers will be trained.
    """
    def __init__(self, backbone, in_features, num_classes=10,
                 num_fc_layers=2, hidden_size=256):
        super().__init__()
        self.backbone = backbone

        layers = []
        input_dim = in_features

        
        for _ in range(max(0, num_fc_layers - 1)):
            layers.append(nn.Linear(input_dim, hidden_size))
            layers.append(nn.ReLU())
            input_dim = hidden_size

        
        layers.append(nn.Linear(input_dim, num_classes))

        self.classifier = nn.Sequential(*layers)

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x



model = PretrainedFashionNet(
    backbone=base_model,
    in_features=backbone_in_features,
    num_classes=10,      
    num_fc_layers=3,     # number of fully connected layers (changed between 1 and 3)
    hidden_size=256
).to(device)

# only training the new fully connected layers
for param in model.classifier.parameters():
    param.requires_grad = True

optimizer = Adam(model.classifier.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()


#training loop 

num_epochs = 3 
train_accuracies = []
test_accuracies = []

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm.tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_accuracies.append(train_acc)

    # evaluating on test set
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)

    test_acc = correct_test / total_test
    test_accuracies.append(test_acc)

    print(
        f"Epoch {epoch + 1}/{num_epochs} - "
        f"Train Loss: {total_loss:.4f}, Train Acc: {train_acc:.2%}, "
        f"Test Acc: {test_acc:.2%}"
    )

end_time = time.time()
total_time = end_time - start_time

print(f"\nTotal training time for Experiment 8: {total_time:.2f} seconds")
print(f"Final Test Accuracy for Experiment 8: {test_accuracies[-1]:.4f}")


Epoch 1/3: 100%|██████████████████████████████| 938/938 [03:24<00:00,  4.59it/s]


Epoch 1/3 - Train Loss: 1262.8518, Train Acc: 50.80%, Test Acc: 60.95%


Epoch 2/3: 100%|██████████████████████████████| 938/938 [03:37<00:00,  4.31it/s]


Epoch 2/3 - Train Loss: 1123.4619, Train Acc: 56.32%, Test Acc: 62.82%


Epoch 3/3: 100%|██████████████████████████████| 938/938 [03:26<00:00,  4.54it/s]


Epoch 3/3 - Train Loss: 1087.3381, Train Acc: 57.66%, Test Acc: 62.70%

Total training time for Experiment 8: 722.31 seconds
Final Test Accuracy for Experiment 8: 0.6270
